<a href="https://colab.research.google.com/github/sayanbanerjee32/feedback-prize-effectiveness/blob/main/all_text_concat_DBERTA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try: import fastkaggle
except ModuleNotFoundError:
    !pip install -Uq fastkaggle

!pip install -q datasets
!pip install transformers
!pip install sentencepiece 
from fastkaggle import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import output
output.enable_custom_widget_manager()

In [3]:
import os
from pathlib import Path
import pandas as pd

In [4]:
# config depending on whether this is running on kaggle or collab
# is_colab = True
is_colab = not os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
comp = 'feedback-prize-effectiveness'
if is_colab:
    model_save_path = Path('/content/'+comp+'_out/models')
else:
    model_save_path = Path('/kaggle/working/'+comp) #+'/models')

In [5]:
# import colab libraries
if is_colab:
    from google.colab import drive
    drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so lets move it there.
if is_colab:
    !mkdir ~/.kaggle
    !cp /content/drive/MyDrive/Kaggle_api_auth/kaggle.json ~/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [7]:
# This permissions change avoids a warning on Kaggle tool startup.
if is_colab:
    !chmod 600 ~/.kaggle/kaggle.json

In [8]:
path = setup_comp(comp)
path

Path('feedback-prize-effectiveness')

## Text classification

In [9]:
def file_read(file_path):
    with open(file_path, 'r') as _f: 
        all_content = _f.read()
    return all_content

In [10]:
# pre-process text - add all columns 
df = pd.read_csv(path/'train.csv')
df['essay_text'] = df['essay_id'].apply(lambda x: file_read(path / 'train' / f'{x}.txt'))
df.head()

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,essay_text
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate,"Hi, i'm Isaac, i'm going to be writing about h..."
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate,"Hi, i'm Isaac, i'm going to be writing about h..."
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate,"Hi, i'm Isaac, i'm going to be writing about h..."
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate,"Hi, i'm Isaac, i'm going to be writing about h..."
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate,"Hi, i'm Isaac, i'm going to be writing about h..."


In [11]:
# See sequence length
df['seq_length_essay'] = [len(txt.split()) for txt in df['essay_text'].tolist()]
df['seq_length_dis'] = [len(txt.split()) for txt in df['discourse_text'].tolist()]
df['seq_length_essay'].describe(), df['seq_length_dis'].describe()

(count    36765.000000
 mean       458.588522
 std        220.423420
 min        144.000000
 25%        288.000000
 50%        408.000000
 75%        579.000000
 max       1367.000000
 Name: seq_length_essay, dtype: float64, count    36765.000000
 mean        44.654073
 std         46.669682
 min          1.000000
 25%         16.000000
 50%         28.000000
 75%         57.000000
 max        836.000000
 Name: seq_length_dis, dtype: float64)

In [12]:
# replace discorse text within the context text as __MASKED__
df['masked_ess_txt'] = df[['essay_text','discourse_text']].apply(lambda row: row.essay_text.strip().replace(row.discourse_text.strip(),
                                                                                                         '__MASKED__'),
                                                              axis = 1)
df['seq_length_mask_ess'] = [len(txt.split()) for txt in df['masked_ess_txt'].tolist()]
df['masked_ess_txt'].head(), df['seq_length_mask_ess'].describe()

(0    __MASKED__ On my perspective, I think that the...
 1    Hi, i'm Isaac, i'm going to be writing about h...
 2    Hi, i'm Isaac, i'm going to be writing about h...
 3    Hi, i'm Isaac, i'm going to be writing about h...
 4    Hi, i'm Isaac, i'm going to be writing about h...
 Name: masked_ess_txt, dtype: object, count    36765.000000
 mean       414.791024
 std        213.788307
 min          1.000000
 25%        254.000000
 50%        367.000000
 75%        531.000000
 max       1345.000000
 Name: seq_length_mask_ess, dtype: float64)

In [13]:
# function to truncate discourse text and context text
# this is still in progress
# def trunc_text(text, num_words, unique_centre_tok = None):
#     w_l = text.split()
#     if unique_centre_tok is None:
#         if len(w_l) > num_words: w_l = w_l[:num_words]
#     else:
#         if len(w_l) > num_words:
#             try: pos_tok = w_l.index(unique_centre_tok) + 1
#             except: # in case there is an issue with the replacement
#                 print(text)
#                 pos_tok = round(num_words / 2)
#             if pos_tok > round(len(w_l) / 2):
#                 if len(w_l) > pos_tok + round((num_words - 1) / 2):
#                     start_pos = pos_tok - round((num_words - 1) / 2) - 1
#                 else:
#                     start_pos = len(w_l) - num_words
#                 w_l = w_l[start_pos:(start_pos + num_words)]
#             else:
#                 if pos_tok > round((num_words -1) / 2):
#                     start_pos = pos_tok - round((num_words -1) / 2) - 1
#                     w_l = w_l[start_pos:(start_pos + num_words)]
#                 else:
#                     w_l = w_l[:num_words]
            
#     return ' '.join(w_l)

# [trunc_text("let's see where __MASKED__ we are going.", i, unique_centre_tok = "__MASKED__") for i in range(2,8)]

In [14]:
# combine all text columns for classification 
# concat all
# df['all_text'] = 'CONTEXT: ' + df.essay_text + '; DISCOURSE: ' + df.discourse_text + '; TYPE: ' + df.discourse_type

# concat after truncation
# df['essay_text_trunc'] = df.masked_ess_txt.apply(lambda t: trunc_text(t,512, "__MASKED__"))
# df['discourse_text_trunc'] = df.discourse_text.apply(lambda t: trunc_text(t,64))
# df['all_text'] = 'CONTEXT: ' + df.essay_text_trunc + '; TYPE: ' + df.discourse_type + '; DISCOURSE: ' + df.discourse_text_trunc 
df['all_text'] = 'CONTEXT: ' + df.masked_ess_txt + '; TYPE: ' + df.discourse_type + '; DISCOURSE: ' + df.discourse_text
df['seq_length_all'] = [len(txt.split()) for txt in df['all_text'].tolist()]
df['seq_length_all'].describe()

count    36765.000000
mean       463.536244
std        220.382475
min        149.000000
25%        293.000000
50%        413.000000
75%        584.000000
max       1373.000000
Name: seq_length_all, dtype: float64

In [15]:
# create data datasets
from datasets import Dataset,DatasetDict
ds = Dataset.from_pandas(df)
ds

Dataset({
    features: ['discourse_id', 'essay_id', 'discourse_text', 'discourse_type', 'discourse_effectiveness', 'essay_text', 'seq_length_essay', 'seq_length_dis', 'masked_ess_txt', 'seq_length_mask_ess', 'all_text', 'seq_length_all'],
    num_rows: 36765
})

In [16]:
model_nm = 'microsoft/deberta-v3-small'

In [17]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer
tokz = AutoTokenizer.from_pretrained(model_nm)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.7/dist-packages/transformers/convert_slow_tokenizer.py:435: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [18]:
def tok_func(x): return tokz(x["all_text"])
tok_ds = ds.map(tok_func, batched=True)

  0%|          | 0/37 [00:00<?, ?ba/s]

In [19]:
dds = tok_ds.train_test_split(0.25, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['discourse_id', 'essay_id', 'discourse_text', 'discourse_type', 'discourse_effectiveness', 'essay_text', 'seq_length_essay', 'seq_length_dis', 'masked_ess_txt', 'seq_length_mask_ess', 'all_text', 'seq_length_all', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27573
    })
    test: Dataset({
        features: ['discourse_id', 'essay_id', 'discourse_text', 'discourse_type', 'discourse_effectiveness', 'essay_text', 'seq_length_essay', 'seq_length_dis', 'masked_ess_txt', 'seq_length_mask_ess', 'all_text', 'seq_length_all', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9192
    })
})

In [20]:
from transformers import TrainingArguments,Trainer

In [21]:
bs = 4
epochs = 4
lr = 0.01

args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1,
                         lr_scheduler_type='cosine', fp16=True,
                            evaluation_strategy="epoch", per_device_train_batch_size=bs,
                          per_device_eval_batch_size=bs*2,
                            num_train_epochs=epochs, weight_decay=0.01, report_to='none')

In [22]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokz)

Some weights of the model checkpoint at microsoft/deberta-v3-small were not used when initializing DebertaV2ForSequenceClassification: ['mask_predictions.dense.weight', 'mask_predictions.dense.bias', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.classifier.bias', 'mask_predictions.LayerNorm.bias', 'mask_predictions.LayerNorm.weight']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from 

In [23]:
trainer.train();

The following columns in the training set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: essay_text, seq_length_essay, essay_id, seq_length_all, seq_length_mask_ess, all_text, discourse_effectiveness, discourse_type, discourse_text, seq_length_dis, discourse_id, masked_ess_txt. If essay_text, seq_length_essay, essay_id, seq_length_all, seq_length_mask_ess, all_text, discourse_effectiveness, discourse_type, discourse_text, seq_length_dis, discourse_id, masked_ess_txt are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 27573
  Num 

ValueError: ignored